In [1]:
import numpy as np ## version 1.17
import pandas as pd 
import os
import json
import re
import math
pd.set_option('display.max_columns', None)

In [2]:
!pip install pickle5
import pickle5

     |████████████████████████████████| 133kB 6.8MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp36-cp36m-linux_x86_64.whl size=218599 sha256=3899b3ef98e7bf2b0c9dc24c5cabd7645026ced79e6ec496feb714984fe6f6a2
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
## Couldn't import the pickle files so using the dataset directly ##
review = pd.read_csv('/N/u/abbane/Carbonate/Downloads/yelp_academic_dataset_review.csv')
business = pd.read_csv('/N/u/abbane/Carbonate/Downloads/yelp_academic_dataset_business.csv')
users = pd.read_csv('/N/u/abbane/Carbonate/Downloads/yelp_academic_dataset_user.csv', encoding="utf-8")

In [ ]:
business = business[business.columns.drop(list(business.filter(regex='attributes')))]

In [ ]:
### preprocessing from Vidit's code since I couldnt import pickle files ##

doc_result = docx2python("/N/u/abbane/Carbonate/Search/Categories.docx")
categories_to_subcategories = {}
k = None
for i in doc_result.body[0][0][0]:
    if len(re.findall('\t', i)) == 0:
        k = ' '.join(re.findall(r'\w+', i))
        categories_to_subcategories[k] = [k]
    
    categories_to_subcategories[k].append(' '.join(re.findall(r'\w+', i)))

subcategories_to_categories = {}
for k, v in categories_to_subcategories.items():
    for x in set(v):
        subcategories_to_categories.setdefault(x.lower(), []).append(k.lower())


# https://stackoverflow.com/questions/54481198/python-match-multiple-substrings-in-a-string
def clean_categories(string):
    if type(string) is str:
        categories = []
        for key, values in categories_to_subcategories.items():
            pattern = r'\b({})\b'.format('|'.join(map(re.escape, values)))
            matches = set(map(str.lower, re.findall(pattern, string, re.IGNORECASE)))
            categories.extend(list(filter(None, [x.lower() for x in values if x.lower() in matches])))
        return list(set(categories))
    else:
        return string




#business = pd.read_csv('yelp_dataset/pandas_business_1.csv')

# Dropping hours and attributes as their sub dictionaries are already added as features.
#business.drop(['Unnamed: 0', 'hours', 'attributes'], axis=1, inplace=True)
business = business[business['is_open'] == 1]

business['rating_category'] = business['stars'].apply(lambda x: 'low' if x < 3 else ('medium' if x < 4 else 'high'))

business = business[business['state'].isin(business.groupby(by='state')['business_id'].count().sort_values(ascending=False).nlargest(7).keys().values)]

business['cleaned_categories'] = business['categories'].apply(clean_categories)

In [ ]:
business['cleaned_categories'] = business['cleaned_categories'].apply(lambda x: x if type(x) is list else math.nan)
business['parent_categories'] = business['cleaned_categories'].apply(lambda x: list(set([subcategories_to_categories[k][0] for k in x])) if type(x) is list else x)

selected_parent_categories = business[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
    by='parent_categories')['business_id'].count().nlargest(2).index.values

In [ ]:
business = business[business['business_id'].isin(
    pd.concat([business[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
        by='parent_categories').get_group(name)['business_id'] for name in selected_parent_categories]))]

In [ ]:
## User data.. need to figure out utf8 encoding..need to fix this  ##
#users['year'] = pd.DatetimeIndex(pd.to_datetime(users['yelping_since'], format='%Y-%m-%d %H:%M:%S')).year
#users = users[users['year'] >= 2010]

In [ ]:
review = review[['user_id','business_id','stars']]
business = business[['business_id','name','city','stars']]

user_review = pd.merge(review,business,how='inner',on='business_id')
user_review.columns = ['user_id','business_id','user_stars','name','city','business_stars']

## Saving the file to reuse it later ##
user_review.to_csv('/N/u/abbane/Carbonate/Downloads/user_review_mat_food_rest.csv')

## Importing preprocessed files directly

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
### Import user reviews directly instead of business and reviews separetly everytime ###
# user_review = pd.read_csv('/N/u/abbane/Carbonate/Downloads/user_review_mat.csv')
# user_review = pd.read_csv("/content/drive/My Drive/archive/user_review_mat_food_rest_v2.csv/")
with open('/content/drive/My Drive/archive/user_review_mat_v3.pkl','rb') as file:
    user_review = pickle5.load(file)

In [5]:
## Top businesses overall ##
user_review.groupby(['business_id','name'])['user_stars'].mean().sort_values(ascending=False).head()

business_id             name                 
T6rvTatc7QQMd-uvDr5OZQ  Headfoneshop             5.0
Jzd5ZpOxzjG4_pxX6C6rBQ  Las Vegas Electrician    5.0
VVxf_sOaHqLzFLDMDYUpVA  Quick Stop               5.0
VTOvXMJvmG1LNmYMdQ3eNw  Q. Contrary              5.0
JyTMXTaqQ1WepVH5Fuv6Aw  DePry Acupuncture        5.0
Name: user_stars, dtype: float64

In [6]:
#user_review.pivot_table(values='user_stars', index='user_id', columns='name', fill_value=0).head()
## sorting the user reviews so I can break ot down for pivot ##
user_review = user_review.sort_values('name')

In [7]:
## remove users having less than 50 ratings to avoid coldstarts ##
## need to try different 'n' and check accuracy ##
users = user_review[['user_id','business_id']].groupby('user_id').count().sort_values(by='business_id',ascending=False).reset_index()
users = users[users.business_id >= 50]

In [8]:
## joining with the user_review table to filter out the users with more than 50 reviews ##
user_review = pd.merge(user_review,users['user_id'],how='inner', on='user_id')

## Trying the surprise library

In [9]:
!pip install surprise

     |████████████████████████████████| 11.8MB 6.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670911 sha256=433a26a566bb6053b0b939f31fae5f66b39f58269b516192c268e9b02eae035d
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [27]:
from surprise import SVD
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

In [11]:
def get_top_n(predictions, n=10):
    """
    This function returns the top n predictions for every user
    
    Parameters:
    
    predictions: ratings for every user predicted by the model
     
    n: Number of restaurants to be recommended default is 10
    
    Returns:
    A dictionary with key as the user id and a list of businesses as the value
    """
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [12]:
def precision_recall_at_k(predictions, k=10, threshold=3):
    """
    This function calculates precision and recall for the predicted ratings of all the users.
    
    Parameters: 
    
    predictions: ratings for every user predicted by the model
    
    k: Value for k in Precision@k / Recall@k, to be evaluated for top k recommendations
    
    threshold: threshold to classify a review as positive or negative, default is 3
    
    Returns:
    Two dictionaries one for precision and recall each. Both the metrics are evaluated at a user level.
    """
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [13]:
## can improve this function  ##
def calculate_ndcg(prediction,k, threshold):
    """
    This function calculates binary NDCG for the predicted ratings of all the users. To use the NDCG in binary form, 
    we would be considering all the ratings below the threshold as non-relevant and all the ratings above the threshold
    as relevant. NDCG will be calculated as:
    (relevancy)/log2(position+1)
        where relevancy can be 1 or 0
    Parameters: 
    
    prediction: ratings for every user predicted by the model
    
    k: Value for k in Precision@k / Recall@k, to be evaluated for top k recommendations
    
    threshold: threshold to classify a review as positive or negative, default is 3
    
    Returns:
    A dictionary one for NDCG which is calculated at a user level.
    """   
    user_est_true = defaultdict(list)
    for uid, _, act_rating, pred_rating, _ in predictions:
        user_est_true[uid].append((pred_rating, act_rating))
    dcg = dict()
    ## DCG
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        terms = []
        pos = 1
        for (pred,act) in user_ratings[:k]:
            if ((pred >= threshold) and (act >= threshold)):
                terms.append(1/np.log2(pos+1))
                pos = pos+1
            else:
                terms.append(0)
                pos = pos+1
        dcg[uid] = sum(terms)
    #IDCG
    idcg = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        
        terms = []
        pos = 1
        for (pred,act) in user_ratings[:k]:
            if ((act >= threshold)):
                terms.append(1/np.log2(pos+1))
                pos = pos+1
            else:
                terms.append(0)
                pos = pos+1
        idcg[uid] = sum(terms)
    ndcg = dict()
    for uid in idcg.keys():
      ndcg[uid] = dcg[uid]/idcg[uid]
    return ndcg

In [14]:
def calculate_mrr(prediction,k, threshold=3):
  """
  This function calculates the mean reciprocal rank for the predicted ratings of all the users. 
  Reciprocal rank is the reciprocal of the position at which the first positive recommendation occurs.
  For example, if the first recommended restaurant was rated positive by the user, MRR is 1 and if
  first recommende was negative but second recommended was positive, then MRR = 1/2 = 0.5
  Parameters: 

  prediction: ratings for every user predicted by the model

  k: Value for k in MRR@K, to be evaluated for top k recommendations

  threshold: threshold to classify a review as positive or negative, default is 3

  Returns:
  A list of Reciprocal rank values at user level.
  """    
  user_est_true = defaultdict(list)
  for uid, _, act_rating, pred_rating, _ in predictions:
      user_est_true[uid].append((pred_rating, act_rating))
  reci_rank=[]
  for uid, user_ratings in user_est_true.items():
      # Sort user ratings by estimated value
      user_ratings.sort(key=lambda x: x[0], reverse=True)
      for idx,(pred,act) in enumerate(user_ratings[:k]):
          if ((pred >= threshold) and (act >= threshold)):
              reci_rank.append(1/(idx+1))
              break
          else:
              continue
  return reci_rank

In [23]:
## precision, recall  and NDCG at k ##
def display_all_metrics(predictions,k):
    """
    This function prints all the evaluation metrics.
    
    Parameters: 
    
    predictions: ratings for every user predicted by the model
    
    k: Value for k in Precision@k / Recall@k, to be evaluated for top k recommendations
    
    Returns:
    Prints Precision, Recall and NDCG for the provided value of k.
    """
    precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=3)
    # ndcg = calculate_ndcg(predictions,k=k,threshold=3)
    mrr = calculate_mrr(predictions,k=k,threshold=3)
    ## mean precision over all users ##
    print('Precision at ',k,':',sum(prec for prec in precisions.values()) / len(precisions))
    ## mean recall over all users ##
    print('Recall at ',k,':',sum(rec for rec in recalls.values()) / len(recalls))
    ## mean NDCG? ##
    # print('NDCG at ',k,':',sum(val for val in ndcg.values()) / len(ndcg))
    ## mean reciprocal rank ##
    print('MRR at ',k,':',np.mean(np.array(mrr)))    

## Base model
The train_test_split divides data into test and train but instead of selecting all the ratings of a user, random number of ratings are used as test data.
The baseline models only check the user and business biases ie. how far are the user or business ratings from the average

In [28]:
## using a pandas dataframe ##
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_review[['user_id', 'business_id', 'user_stars']], reader)

trainset, testset = train_test_split(data, test_size=.20)
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

# Train the algorithm on the trainset, and predict ratings for the testset
# we can add cross validation later on #
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...


In [29]:
print('RMSE: ',accuracy.rmse(predictions))
display_all_metrics(predictions,k=5)
display_all_metrics(predictions,k=10)

RMSE: 1.0088
RMSE:  1.0087571780681397
Precision at  5 : 0.9180761574520186
Recall at  5 : 0.36447005704903945
MRR at  5 : 0.9718800439021241
Precision at  10 : 0.8961652188005568
Recall at  10 : 0.6524122975766956
MRR at  10 : 0.9717241156726053


In [30]:
## using a pandas dataframe ##
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_review[['user_id', 'business_id', 'user_stars']], reader)

trainset, testset = train_test_split(data, test_size=.20)
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)

# Train the algorithm on the trainset, and predict ratings for the testset
# we can add cross validation later on #
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using sgd...


In [31]:
print('RMSE: ',accuracy.rmse(predictions))
display_all_metrics(predictions,k=5)
display_all_metrics(predictions,k=10)

RMSE: 1.1110
RMSE:  1.1109622997056654
Precision at  5 : 0.9173120143838889
Recall at  5 : 0.37021841213215906
MRR at  5 : 0.9697545144913566
Precision at  10 : 0.889247721177992
Recall at  10 : 0.6695333809495542
MRR at  10 : 0.9694296848950925


# SVD with Stochastic Gradient Descent

In [24]:
## using a pandas dataframe ##
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_review[['user_id', 'business_id', 'user_stars']], reader)

trainset, testset = train_test_split(data, test_size=.20)
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
# we can add cross validation later on #
algo.fit(trainset)
predictions = algo.test(testset)

In [25]:
print('RMSE: ',accuracy.rmse(predictions))
display_all_metrics(predictions,k=5)
display_all_metrics(predictions,k=10)

RMSE: 1.0232
RMSE:  1.0231571941956277
Precision at  5 : 0.9126565209015799
Recall at  5 : 0.35937139116992023
MRR at  5 : 0.9719112406867509
Precision at  10 : 0.8914748664507952
Recall at  10 : 0.6441468140192236
MRR at  10 : 0.9711248266296809


## Using K-fold CV

In [ ]:
## Using k-fold cross validations ##
#data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()
n = 1
for trainset, testset in kf.split(data):
    print('Fold: ',n)
    algo.fit(trainset)
    predictions_kfold = algo.test(testset)
    accuracy.rmse(predictions_kfold)
    display_all_metrics(predictions_kfold,k=15)
    display_all_metrics(predictions_kfold,k=10)
    n=n+1

In [ ]:
## Top 10 recommendations for every user ##
top_n = get_top_n(predictions, n=10)
## Top 10 from k-fold ##
top_n_kfold = get_top_n(predictions_kfold, n=10)

## GridSearchCV for getting the best parameters

In [ ]:
param_grid = {'n_epochs': [5, 10, 15, 25], 'lr_all': [0.0005, 0.002, 0.005, 0.01, 0.08, 0.1],
              'reg_all': [0.01, 0.08, 0.1, 0.5, 0.9]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

In [ ]:
gs.fit(data)

In [ ]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

## Testing with an anti-test data
Anti-test data would be the businesses/restaurants that a user has not rated yet. Example, if out of 100 restaurants, a user has rated 20, then the ant-test dataset would include the other 80 restaurants.

In [ ]:
## using a pandas dataframe ##
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_review[['user_id', 'business_id', 'user_stars']], reader)

trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Testing on anti-test data
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
top_n_antitest = get_top_n(predictions,10)